In [1]:
import numpy as np
from random import random
import multiprocessing as mp
from itertools import repeat

N = 10**6

lst = [random() for i in range(N)]

In [2]:
num_arr = np.random.randn(N)

In [3]:
%%timeit
np.sum(num_arr)

1.19 ms ± 13.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## In Serial

In [3]:
def serial_mean(lst):
    return sum(lst)/len(lst)
def serial_sd(lst, u=None):
    if u==None:
        u = serial_mean(lst)
    return sum([(x-u)**2 for x in lst])/(len(lst) - 1)

In [4]:
%%timeit
serial_mean(lst)

12.2 ms ± 93.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%%timeit
serial_mean(num_arr)

80.5 ms ± 406 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
u_serial = serial_mean(lst)
sd_serial = serial_sd(lst, u_serial)

## In Serial with C-like loops

In [ ]:
%%timeit
my_sum = 0
for n in lst:
    my_sum+=n
u_serial = my_sum/len(lst)

## In Parallel

In [6]:
def parallel_mean(lst, p):
    pool = mp.Pool(processes = p)
    l = len(lst)
    size = l//p + 1
    my_lst = [lst[i:i + size] for i in range(0, l, size)]
    my_lst = pool.map(sum, my_lst)
    pool.close()
    pool.join()
    return sum(my_lst)/len(lst)
    

In [ ]:
%%timeit
parallel_mean(lst,1) - u_serial

In [9]:
%%timeit
parallel_mean(lst,4) - u_serial

123 ms ± 611 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit
parallel_mean(num_arr,4) - u_serial

122 ms ± 812 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
def proc_sum2(args):
    lst, i, size = args[0], args[1], args[2]
    return sum(lst[i*size:(i+1)*size])

def parallel_mean2(lst,p):
    manager = mp.Manager()
    shared_list = manager.list(lst)
    pool = mp.Pool(processes = p)
    l = len(lst)
    size = l//p + 1
    sums = pool.map(proc_sum2, [(shared_list, i,size) for i in range(p)])
    pool.close()
    pool.join()
    return sum(sums)/l
    
    

In [ ]:
%%timeit
parallel_mean2(lst,1) - u_serial

In [ ]:
%%timeit
parallel_mean2(lst,4) - u_serial

In [ ]:
def proc_sum3(args):
    lst, i, size = args[0], args[1], args[2]
    my_sum = 0
    for num in lst[i*size:(i+1)*size]:
        my_sum+= num
    return sum(lst[i*size:(i+1)*size])

def parallel_mean3(lst,p):
    manager = mp.Manager()
    shared_list = manager.list(lst)
    pool = mp.Pool(processes = p)
    l = len(lst)
    size = l//p + 1
    sums = pool.map(proc_sum3, [(shared_list, i,size) for i in range(p)])
    pool.close()
    pool.join()
    return sum(sums)/l

In [ ]:
%%timeit
parallel_mean3(lst,1) - u_serial

In [ ]:
%%timeit
parallel_mean3(lst,4) - u_serial

In [ ]:
def proc_sum4(i, size, lst, ret_list):
    #lst, i, size = args[0], args[1], args[2]
    my_sum = 0
    for num in lst[i*size:(i+1)*size]:
        my_sum+= num
    ret_list[i] = sum(lst[i*size:(i+1)*size])
    return 

def parallel_mean4(lst,p):
    manager = mp.Manager()
    shared_list = manager.list(lst)
    ret_list = manager.list([0]*p)
    #pool = mp.Pool(processes = p)
    l = len(lst)
    size = l//p + 1
    jobs = [mp.Process(target=proc_sum4, args=(i, size, shared_list, ret_list))
            for i in range(p)]
    #sums = pool.map(proc_sum, [(shared_list, i,size) for i in range(p)])
    for job in jobs: job.start()
    for job in jobs: job.join()
    return sum(ret_list)/l

In [ ]:
%%timeit
parallel_mean4(lst,1) - u_serial

In [ ]:
%%timeit
parallel_mean4(lst,4) - u_serial

In [ ]:
def proc_sum3(args):
    lst, i, size = args[0], args[1], args[2]
    my_sum = 0
    for num in lst[i*size:(i+1)*size]:
        my_sum+= num
    return sum(lst[i*size:(i+1)*size])

def parallel_mean3(lst,p):
    manager = mp.Manager()
    shared_list = manager.Array('f',lst)
    pool = mp.Pool(processes = p)
    l = len(lst)
    size = l//p + 1
    sums = pool.map(proc_sum3, [(shared_list, i,size) for i in range(p)])
    pool.close()
    pool.join()
    return sum(sums)/l

In [ ]:
%%timeit
parallel_mean5(lst,1) - u_serial